In [3]:
import pathlib
import pandas as pd
import numpy as np
import re
import time
import fasttext
fasttext.FastText.eprint = lambda x: None
from huggingface_hub import hf_hub_download
from tqdm import tqdm

In [6]:
def get_lang(
    df: pd.DataFrame,
    col_calculate_on: str
) -> str:
    """Detects the language of a text column in a DataFrame using langdetect.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the text column
    col_calculate_on : str
        Column name to calculate language on

    Returns
    -------
    df : pd.DataFrame
        DataFrame with the language column added to it as "lang"
    """

    def det(x: str) -> str:
        """
        Detects the language of a given text

        Parameters
        ----------
        x : str
            Text whose language is to be detected

        Returns
        -------
        lang : str
            Language of the text
        """

        try:
            lang = model.predict(x)[0][0].replace('__label__','')
        except Exception as e:
            print(e)
            lang = 'Other'
        return lang

    print(f"-- Detecting language...")
    start_time = time.time()

    df['lang'] = df[col_calculate_on].apply(det)

    print(f'-- -- Language detect finished in {(time.time() - start_time)}')

    return df

def normalize_string(s):
    # Convert to lower case
    #s = s.lower()
    # Remove punctuation and special characters
    #s = re.sub(r'[^\w\s]', '', s)
    # Remove extra spaces
    s = re.sub(r'\s+', ' ', s).strip()
    return s

Estos ficheros se generan con el notebook "Procesado_Textos_BSC".

No obstante, algo no cuadra: 
- Montse dijo que había **155.980** documentos en **catalán**
- De los documentos que nos pasan, sólo aparecen **3.942** (esto lo obtenemos si leemos ```/export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated``` y hacemos ``combined_df_ca[(combined_df_ca['translated_content'] != "") & (pd.notna(combined_df_ca['translated_content']))]`` o ``combined_df_ca[combined_df_ca.alternative_lang == "ca"]``. 
- Si nos quedamos con los que son ténicos, sólo quedan **839** documentos traducidos del catalán. ¿No son pocos?


**Repuesta de BSC de cómo detectan el lenguaje:** Para la identificación de idiomas de nextProcurement usamos el modelo de Fasttext: https://fasttext.cc/docs/en/language-identification.html

En nuestra implementación, ya que el modelo solo puede clasificar pequeñas porciones de texto, no clasificamos el documento entero directamente, si no que vamos frase por frase.

Entonces, para cada frase en el documento, obtenemos el idioma con mayor probabilidad, calculada por el modelo. Con las frases clasificadas por idiomas, calculamos el porcentaje de carácteres en cada idioma. Finalmente, clasificamos el documento con la lengua que tenga más del 60% de carácteres. Cuando no hay ninguna lengua que sobrepase este 60%, los documentos se clasifican como "mx". En "mx" es normal encontrar documentos con lenguas minoritarias que el modelo no sabe clasificar bien (gallego, catalán) y que además contienen código o puntuación excesiva.

**Nosotros de momento vamos a aplicar fastext en el texto sin más, a la espera de que BSC nos pase los idiomas que ellos han detectado.**

Han hecho algo raro con los nuevos parquets que nos han pasado traducidos. Al hacer el extract de los que no han sido traducidos, muchos aparecen vacíos. Seguramente se puede resolver el fallo revisando el código de extracción, pero de momento vamos a trabajar haciendo un merge de los que ya teníamos bien procesado del dump anterior (``BSC_procesados``) con los nuevos procesados tras la traducción (``BSC_procesados_ca``).

In [4]:
path_all = pathlib.Path("/export/data_ml4ds/NextProcurement/PLACE/BSC_procesados") # path to all
path_ca_tr = pathlib.Path("/export/data_ml4ds/NextProcurement/PLACE/BSC_procesados_ca") # path to detected as Catalan and translated #BSC_procesados_ca #pdf2txt_ca_translated

In [5]:
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path) 

In [7]:
# Read ALL
paths = []
paths = [path for path in path_all.iterdir()]
all_dfs = []
for path_ in tqdm(paths):
    df = pd.read_parquet(path_)
    all_dfs.append(df)
combined_df = pd.concat(all_dfs)

100%|██████████| 17/17 [19:14<00:00, 67.92s/it]


In [8]:
combined_df

,procurement_id,doc_name,extracted,extracted_tags,texto_heading,embeddings_heading
2,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297..."
10,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051..."
23,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05..."
35,ntp00008052,ntp00008052_Pliego_Prescripciones_tecnicas_URI,íHJJ Agencia de I'Habitatge\n FUU de Catalunya...,"[document, section, heading, section, heading,...",[íHJJ Agencia de I'Habitatge\n FUU de Cataluny...,"[[0.050423447, 0.10861048, 0.12632288, 0.06832..."
40,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294..."
...,...,...,...,...,...,...
250028,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204..."
250038,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086..."
250067,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,ANEX\nPliego de prescripciones técnicas que ha...,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037..."
250078,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800..."


In [9]:
# Read CA
paths = []
paths = [path for path in path_ca_tr.iterdir()]
all_dfs = []
for path_ in tqdm(paths):
    df = pd.read_parquet(path_)
    all_dfs.append(df)
combined_df_ca = pd.concat(all_dfs)

100%|██████████| 17/17 [05:06<00:00, 18.01s/it]


In [10]:
combined_df_ca

,procurement_id,doc_name,extracted,alternative_lang,translated
2,ntp01312172,ntp01312172_Pliego_Prescripciones_tecnicas_URI,3ena\n Aeropuerto de Santiago-Rosalia de Castr...,,no
10,ntp00943963,ntp00943963_Pliego_Prescripciones_tecnicas_URI,"JUNTA DE EXTREMADURA\n , Vicepresidencia Segun...",,no
25,ntp00302546,ntp00302546_Pliego_Prescripciones_tecnicas_URI,EXP- NÚM. PS COT 2022/12(A-74)\nCONTRACTE DE R...,,no
37,ntp01000028,ntp01000028_Pliego_Prescripciones_tecnicas_URI,EJERCITO DETIERRA\n MINISTERIO MANDO DE PERSON...,,no
41,ntp00578073,ntp00578073_Pliego_Prescripciones_tecnicas_URI,EXPEDIENTE N* 2 0114 18 0065 00\nPLIEGO DE PRE...,,no
...,...,...,...,...,...
251017,ntp00481782,ntp00481782_Pliego_Prescripciones_tecnicas_URI,/ . EXPEDIENTE 2018-24\n GENERALITAT Fundació ...,,no
251029,ntp11581768,ntp11581768_Pliego_Prescripciones_tecnicas_URI,"ÁNGEL MILLÁN DE MIGUEL\n AMIDEMI, S. L. Ingeni...",,no
251034,ntp01341366,ntp01341366_Pliego_Prescripciones_tecnicas_URI,"E\n IS NUCLEARES, S.A., S.M.E. g;\nPLIEGO DE P...",,no
251037,ntp00593148,ntp00593148_Pliego_Prescripciones_tecnicas_URI,DILIGENCIA: Para hacer constar que el presente...,,no


In [11]:
combined_df_ca.columns

Index(['procurement_id', 'doc_name', 'extracted', 'alternative_lang',
       'translated'],
      dtype='object')

In [12]:
combined_df_ca[combined_df_ca.translated == "yes"]

,procurement_id,doc_name,extracted,alternative_lang,translated
168,ntp00334697,ntp00334697_Pliego_Prescripciones_tecnicas_URI,a Ayuntamiento de ““iy” Sant Celoni PLIEGO DE ...,ca,yes
236,ntp00262987,ntp00262987_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Granollers Recursos Humanos 1»...,ca,yes
421,ntp00389750,ntp00389750_Pliego_Prescripciones_tecnicas_URI,Descripción del contrato: Servicio Club de Tra...,ca,yes
430,ntp00286598,ntp00286598_Pliego_Prescripciones_tecnicas_URI,PROYECTO DE REFORMA DE LOS VESTUARIOS DE LA PI...,ca,yes
525,ntp01099657,ntp01099657_Pliego_Prescripciones_tecnicas_URI,G CONSEJERIA O AFERS SOCIALS Y |DEPORTES B SI ...,ca,yes
...,...,...,...,...,...
91450,ntp00369338,ntp00369338_Pliego_Prescripciones_tecnicas_URI,IN Generalidad de Cataluña NUD Departamento de...,ca,yes
92023,ntp00415177,ntp00415177_Pliego_Prescripciones_tecnicas_URI,Ayuntamiento de Granollers Contratación y Comp...,ca,yes
92198,ntp00367712,ntp00367712_Pliego_Prescripciones_tecnicas_URI,"_.,¿s | DS p Q E\% U 7N Q\s PCT_07/22 PLIEGO D...",ca,yes
92235,ntp00283716,ntp00283716_Pliego_Prescripciones_tecnicas_URI,u u u — TRASLADO DE VETERINARIA • DE AVES A LA...,ca,yes


In [13]:
merged_df = pd.merge(combined_df, combined_df_ca, on='doc_name', how='inner')

merged_df['extracted'] = merged_df.apply(
    lambda row: row['extracted_y'] if row['alternative_lang'] == 'ca' else row['extracted_x'], axis=1)

merged_df = merged_df.drop(columns=['extracted_x', 'extracted_y', 'procurement_id_y'])
merged_df

,procurement_id_x,doc_name,extracted_tags,texto_heading,embeddings_heading,alternative_lang,translated,extracted
0,ntp00114200,ntp00114200_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n ...,"[[-0.1855995, 0.10908591, 0.1287027, -0.085297...",,no,g PLIEGO DE PRESCRI PCI ONES TÉCNI CAS\n S\n \...
1,ntp00729859,ntp00729859_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE R...,"[[-0.06488852, 0.03319587, 0.05720649, -0.2051...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS QUE HA DE RE...
2,ntp01335443,ntp01335443_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[0.11980498, -0.0047292123, 0.07927151, -0.05...",,no,PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CONT...
3,ntp00008052,ntp00008052_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, section, heading,...",[íHJJ Agencia de I'Habitatge\n FUU de Cataluny...,"[[0.050423447, 0.10861048, 0.12632288, 0.06832...",,no,íHJJ Agencia de I'Habitatge\n FUU de Catalunya...
4,ntp00196761,ntp00196761_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[Agencia de Vivienda Social\n CONSEJERÍA DE VI...,"[[0.2535557, 0.06023139, 0.13293944, 0.0011294...",,no,Ref: 50/088288.9/21\nAgencia de Vivienda Socia...
...,...,...,...,...,...,...,...,...
455432,ntp00582515,ntp00582515_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[PLIEGO DE PRESCRIPCIONES TÉCNICAS PARA LA CON...,"[[-0.101150624, 0.29571363, -0.01729959, 0.204...",,no,sura\nPLIEGO DE PRESCRIPCIONES TÉCNICAS PARA L...
455433,ntp01351598,ntp01351598_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...","[COLEGIO DE INGENIEROS DE CAMINOS,\n CANALES Y...","[[-0.041178588, -0.10482038, 0.20265633, 0.086...",,no,2020/02293/01 14/07/2020\n \n ...\nCOLEGIO DE ...
455434,ntp00566851,ntp00566851_Pliego_Prescripciones_tecnicas_URI,"[document, section, heading, body, p, p, p, p,...","[ANEX, Características técnicas de la aplicaci...","[[-0.02871781, 0.109616816, -0.03430078, 0.037...",,no,ANEX\nPliego de prescripciones técnicas que ha...
455435,ntp00882149,ntp00882149_Pliego_Prescripciones_tecnicas_URI,"[document, section, body, p, section, heading,...",[UNIVERSITAT\n POLITÉCNICA\n DE VALENCIA\n \n ...,"[[0.04755034, -0.071341686, 0.07659534, 0.1800...",,no,a\n A\n S\nUNIVERSITAT\n POLITÉCNICA\n DE VALE...


In [14]:
merged_df["clean_extracted"] = merged_df["extracted"].apply(normalize_string)
merged_df = get_lang(merged_df, col_calculate_on = "clean_extracted")

-- Detecting language...
-- -- Language detect finished in 10958.432528734207


In [16]:
merged_df.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_29jul.parquet")

In [15]:
sample_df = merged_df.sample(n=1000, random_state=42)
sample_df.to_parquet("/export/data_ml4ds/NextProcurement/PLACE/pliegos_extracted/all_extracted_29jul_sample1000.parquet")